In [1]:
import torch
import numpy as np
import json
import matplotlib.pyplot as plt
import torch.utils.data
import torch.functional as F
from torch.autograd import Variable

In [2]:
with open("train.json", encoding='utf-8') as f:
    line = f.readline()
    train_data = json.loads(line)

with open("test.json", encoding="utf-8") as f:
    line = f.readline()
    test_data = json.loads(line)

In [3]:
audio_data = [each["audio_embedding"] for each in train_data]
label_data = [each["is_turkey"] for each in train_data]

In [4]:
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, audio_data, label_data):
        audio_tensor = [torch.FloatTensor(x) for x in audio_data]
        audio_tensor = torch.nn.utils.rnn.pad_sequence(audio_tensor)
        audio_tensor.transpose_(0, 1)
        self._audio_data = audio_tensor
        self._label_data = torch.FloatTensor(label_data)
    def __getitem__(self, index):
        return self._audio_data[index], self._label_data[index]
    def __len__(self):
        return len(self._audio_data)
    
test_set = [[x["vid_id"], torch.Tensor(x["audio_embedding"])] for x in test_data]

class Mytestset(torch.utils.data.Dataset):
    def __init__(self, audio_data_with_vid):
        self._audio_data = [x[1] for x in audio_data_with_vid]
        self._audio_data = torch.nn.utils.rnn.pad_sequence(self._audio_data)
        self._audio_data.transpose_(0, 1)
        self._vid = [x[0] for x in audio_data_with_vid]
        
    def __getitem__(self, index):
        return self._audio_data[index], self._vid[index]
    
    def __len__(self):
        return len(self._audio_data)    

In [5]:
dataset = Mydataset(audio_data, label_data)
data_size = len(dataset)
train_size = int(0.8 * data_size)
valid_size = data_size - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])
test_dataset = Mytestset(test_set)

train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=len(train_dataset))
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, shuffle=True, batch_size=len(valid_dataset))
all_dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size = len(dataset))
test_dataloader = torch.utils.data.DataLoader(test_dataset)

In [6]:
class globalPool(torch.nn.Module):
    def __init__(self, isAvg):
        super(globalPool, self).__init__()
        self._isAvg = isAvg
    def forward(self, x : torch.Tensor):
        assert x.dim() == 3, "only useful for dim = 3"
        B, S, F = x.size()
        x = x.view(B, -1, S, F)
        if self._isAvg:
            return torch.mean(x, 2)
        else:
            max_pool, _ = torch.max(x, 2)
            return max_pool

class Model(torch.nn.Module):
    def __init__(self, hidden_size=256):
        super(Model, self).__init__()
        self._bn = torch.nn.Sequential(
            torch.nn.BatchNorm1d(num_features=10),
            torch.nn.Dropout(0.5)
        )
        self._LSTM = torch.nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self._classify = torch.nn.Sequential(
            torch.nn.ReLU(),
            # torch.nn.BatchNorm1d(num_features=10),
            # torch.nn.Dropout(0.25),
            torch.nn.Linear(hidden_size * 2, 1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self._bn(x)
        x, _ = self._LSTM(x)
        x = self._classify(x)
        return x
    
class Model_new(torch.nn.Module):
    def __init__(self, hidden_size=256):
        super(Model_new, self).__init__()
        self._bn = torch.nn.BatchNorm1d(num_features=10)
        self._gru1 = torch.nn.LSTM(input_size=128, hidden_size=128, batch_first=True, bidirectional=True)
        self._gru2 = torch.nn.LSTM(input_size=256, hidden_size=64, batch_first=True, bidirectional=True)
        self._avgpool = globalPool(isAvg=True)
        self._maxpool = globalPool(isAvg=False)
        self._fc = torch.nn.Sequential(
            torch.nn.Linear(256, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(64, 1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x : torch.Tensor):
        x = self._bn(x)
        x, _ = self._gru1(x)
        x, _ = self._gru2(x)
        avgp = self._avgpool(x)
        maxp = self._maxpool(x)
        x = torch.cat([avgp, maxp], dim=2)
        x.squeeze_(1)
        x = self._fc(x)
        return x
        

In [29]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")
mymodel = Model_new()
mymodel.to(device)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(mymodel.parameters())
# mymodel = mymodel.double()

In [28]:
epoch = 1000
# def train(epoch):
for i in range(epoch):
    mymodel.train()
    for idx, (data, target) in enumerate(train_dataloader):
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)
        data, target = data.float(), target.float()
        optimizer.zero_grad()
        y_pred = mymodel(data)
        if y_pred.dim() == 3:
            y_pred = y_pred[:, -1, 0]
        elif y_pred.dim() == 2:
            y_pred = y_pred.view(y_pred.size()[0])
        
        loss = criterion(y_pred, target)
        loss.backward()
        optimizer.step()

    mymodel.eval()
    accu = 0
    for idx, (data, target) in enumerate(valid_dataloader):
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)
        data, target = data.float(), target.float()
        y_pred = mymodel(data)
        if y_pred.dim() == 3:
            y_pred = y_pred[:, -1, 0]
        elif y_pred.dim() == 2:
            y_pred = y_pred.view(y_pred.size()[0])
        accu += ((y_pred>0.5).float()==target).sum().item()
    print("Valid mode: accuracy {}/{}".format(accu, len(valid_dataset)))
    print("epoch: {}, loss: {}".format(i, loss.item()))

Valid mode: accuracy 139/239
epoch: 0, loss: 0.7767272591590881
Valid mode: accuracy 144/239
epoch: 1, loss: 0.7713950872421265
Valid mode: accuracy 140/239
epoch: 2, loss: 0.7653337717056274
Valid mode: accuracy 138/239
epoch: 3, loss: 0.7581011056900024
Valid mode: accuracy 137/239
epoch: 4, loss: 0.7491045594215393
Valid mode: accuracy 138/239
epoch: 5, loss: 0.7365739941596985
Valid mode: accuracy 140/239
epoch: 6, loss: 0.7226409316062927
Valid mode: accuracy 140/239
epoch: 7, loss: 0.7085635662078857
Valid mode: accuracy 143/239
epoch: 8, loss: 0.6922146677970886
Valid mode: accuracy 153/239
epoch: 9, loss: 0.677742600440979
Valid mode: accuracy 165/239
epoch: 10, loss: 0.6619834303855896
Valid mode: accuracy 183/239
epoch: 11, loss: 0.6483076810836792
Valid mode: accuracy 201/239
epoch: 12, loss: 0.6346198320388794
Valid mode: accuracy 212/239
epoch: 13, loss: 0.6224756836891174
Valid mode: accuracy 223/239
epoch: 14, loss: 0.6126627922058105
Valid mode: accuracy 223/239
epoch: 

Valid mode: accuracy 227/239
epoch: 134, loss: 0.5534209609031677
Valid mode: accuracy 227/239
epoch: 135, loss: 0.5534253120422363
Valid mode: accuracy 227/239
epoch: 136, loss: 0.5534103512763977
Valid mode: accuracy 227/239
epoch: 137, loss: 0.5534253120422363
Valid mode: accuracy 227/239
epoch: 138, loss: 0.5534262657165527
Valid mode: accuracy 227/239
epoch: 139, loss: 0.553480863571167
Valid mode: accuracy 227/239
epoch: 140, loss: 0.5534260272979736
Valid mode: accuracy 227/239
epoch: 141, loss: 0.5534523129463196
Valid mode: accuracy 227/239
epoch: 142, loss: 0.5534506440162659
Valid mode: accuracy 227/239
epoch: 143, loss: 0.5534339547157288
Valid mode: accuracy 227/239
epoch: 144, loss: 0.5534337759017944
Valid mode: accuracy 227/239
epoch: 145, loss: 0.5534400343894958
Valid mode: accuracy 227/239
epoch: 146, loss: 0.553421139717102
Valid mode: accuracy 227/239
epoch: 147, loss: 0.5534487962722778
Valid mode: accuracy 227/239
epoch: 148, loss: 0.5534502863883972
Valid mode: 

Valid mode: accuracy 224/239
epoch: 261, loss: 0.5523649454116821
Valid mode: accuracy 224/239
epoch: 262, loss: 0.5523682236671448
Valid mode: accuracy 224/239
epoch: 263, loss: 0.5523300170898438
Valid mode: accuracy 224/239
epoch: 264, loss: 0.5523698925971985
Valid mode: accuracy 224/239
epoch: 265, loss: 0.5523545145988464
Valid mode: accuracy 223/239
epoch: 266, loss: 0.552355170249939
Valid mode: accuracy 223/239
epoch: 267, loss: 0.5523120164871216
Valid mode: accuracy 223/239
epoch: 268, loss: 0.5517317056655884
Valid mode: accuracy 222/239
epoch: 269, loss: 0.5517383813858032
Valid mode: accuracy 222/239
epoch: 270, loss: 0.5517281889915466
Valid mode: accuracy 222/239
epoch: 271, loss: 0.5517301559448242
Valid mode: accuracy 222/239
epoch: 272, loss: 0.5517385601997375
Valid mode: accuracy 222/239
epoch: 273, loss: 0.5517250895500183
Valid mode: accuracy 222/239
epoch: 274, loss: 0.5517246723175049
Valid mode: accuracy 222/239
epoch: 275, loss: 0.5517358779907227
Valid mode:

Valid mode: accuracy 223/239
epoch: 390, loss: 0.551723062992096
Valid mode: accuracy 223/239
epoch: 391, loss: 0.5517122149467468
Valid mode: accuracy 223/239
epoch: 392, loss: 0.5517164468765259
Valid mode: accuracy 223/239
epoch: 393, loss: 0.5517178177833557
Valid mode: accuracy 223/239
epoch: 394, loss: 0.5517163276672363
Valid mode: accuracy 223/239
epoch: 395, loss: 0.5517129302024841
Valid mode: accuracy 223/239
epoch: 396, loss: 0.5517247319221497
Valid mode: accuracy 223/239
epoch: 397, loss: 0.5517207980155945
Valid mode: accuracy 223/239
epoch: 398, loss: 0.5517159104347229
Valid mode: accuracy 223/239
epoch: 399, loss: 0.5517147779464722
Valid mode: accuracy 223/239
epoch: 400, loss: 0.5517132878303528
Valid mode: accuracy 223/239
epoch: 401, loss: 0.5517131686210632
Valid mode: accuracy 223/239
epoch: 402, loss: 0.5517145395278931
Valid mode: accuracy 223/239
epoch: 403, loss: 0.5517167448997498
Valid mode: accuracy 223/239
epoch: 404, loss: 0.5517230033874512
Valid mode:

Valid mode: accuracy 223/239
epoch: 523, loss: 0.5517106056213379
Valid mode: accuracy 223/239
epoch: 524, loss: 0.5517114400863647
Valid mode: accuracy 223/239
epoch: 525, loss: 0.5517118573188782
Valid mode: accuracy 223/239
epoch: 526, loss: 0.5517215728759766
Valid mode: accuracy 223/239
epoch: 527, loss: 0.5517126321792603
Valid mode: accuracy 223/239
epoch: 528, loss: 0.5517131090164185
Valid mode: accuracy 223/239
epoch: 529, loss: 0.5517119765281677
Valid mode: accuracy 223/239
epoch: 530, loss: 0.5517106056213379
Valid mode: accuracy 223/239
epoch: 531, loss: 0.5517131090164185
Valid mode: accuracy 223/239
epoch: 532, loss: 0.551709771156311
Valid mode: accuracy 223/239
epoch: 533, loss: 0.5517275333404541
Valid mode: accuracy 223/239
epoch: 534, loss: 0.5517114400863647
Valid mode: accuracy 223/239
epoch: 535, loss: 0.5517101883888245
Valid mode: accuracy 223/239
epoch: 536, loss: 0.5517192482948303
Valid mode: accuracy 223/239
epoch: 537, loss: 0.5517131090164185
Valid mode:

Valid mode: accuracy 223/239
epoch: 653, loss: 0.5517107248306274
Valid mode: accuracy 223/239
epoch: 654, loss: 0.5517093539237976
Valid mode: accuracy 223/239
epoch: 655, loss: 0.551709771156311
Valid mode: accuracy 223/239
epoch: 656, loss: 0.5517095327377319
Valid mode: accuracy 223/239
epoch: 657, loss: 0.5517123341560364
Valid mode: accuracy 223/239
epoch: 658, loss: 0.5517103672027588
Valid mode: accuracy 223/239
epoch: 659, loss: 0.5517100691795349
Valid mode: accuracy 223/239
epoch: 660, loss: 0.5517147183418274
Valid mode: accuracy 223/239
epoch: 661, loss: 0.5517093539237976
Valid mode: accuracy 223/239
epoch: 662, loss: 0.5517128109931946
Valid mode: accuracy 223/239
epoch: 663, loss: 0.5517164468765259
Valid mode: accuracy 223/239
epoch: 664, loss: 0.5517348647117615
Valid mode: accuracy 223/239
epoch: 665, loss: 0.5517103672027588
Valid mode: accuracy 223/239
epoch: 666, loss: 0.5517104864120483
Valid mode: accuracy 223/239
epoch: 667, loss: 0.5517097115516663
Valid mode:

Valid mode: accuracy 223/239
epoch: 785, loss: 0.551708996295929
Valid mode: accuracy 223/239
epoch: 786, loss: 0.5517092943191528
Valid mode: accuracy 223/239
epoch: 787, loss: 0.5517118573188782
Valid mode: accuracy 223/239
epoch: 788, loss: 0.5517088770866394
Valid mode: accuracy 223/239
epoch: 789, loss: 0.5517166256904602
Valid mode: accuracy 223/239
epoch: 790, loss: 0.5517090559005737
Valid mode: accuracy 223/239
epoch: 791, loss: 0.5517116189002991
Valid mode: accuracy 223/239
epoch: 792, loss: 0.5517090559005737
Valid mode: accuracy 223/239
epoch: 793, loss: 0.5517101287841797
Valid mode: accuracy 223/239
epoch: 794, loss: 0.5517131090164185
Valid mode: accuracy 223/239
epoch: 795, loss: 0.5517091751098633
Valid mode: accuracy 223/239
epoch: 796, loss: 0.5517109632492065
Valid mode: accuracy 223/239
epoch: 797, loss: 0.5517094135284424
Valid mode: accuracy 223/239
epoch: 798, loss: 0.5517094731330872
Valid mode: accuracy 223/239
epoch: 799, loss: 0.5517104864120483
Valid mode:

Valid mode: accuracy 223/239
epoch: 910, loss: 0.5517106652259827
Valid mode: accuracy 223/239
epoch: 911, loss: 0.5517106056213379
Valid mode: accuracy 223/239
epoch: 912, loss: 0.5517084002494812
Valid mode: accuracy 223/239
epoch: 913, loss: 0.5517126321792603
Valid mode: accuracy 223/239
epoch: 914, loss: 0.5517106056213379
Valid mode: accuracy 223/239
epoch: 915, loss: 0.5517122745513916
Valid mode: accuracy 223/239
epoch: 916, loss: 0.5517092943191528
Valid mode: accuracy 223/239
epoch: 917, loss: 0.5517077445983887
Valid mode: accuracy 223/239
epoch: 918, loss: 0.551713228225708
Valid mode: accuracy 223/239
epoch: 919, loss: 0.5517104268074036
Valid mode: accuracy 223/239
epoch: 920, loss: 0.5517086386680603
Valid mode: accuracy 223/239
epoch: 921, loss: 0.5517086982727051
Valid mode: accuracy 223/239
epoch: 922, loss: 0.5517085790634155
Valid mode: accuracy 223/239
epoch: 923, loss: 0.5517080426216125
Valid mode: accuracy 223/239
epoch: 924, loss: 0.551708996295929
Valid mode: 

In [30]:
epoch = 1000
mymodel = Model_new()
mymodel.to(device)
for i in range(epoch):
    mymodel.train()
    for idx, (data, target) in enumerate(all_dataloader):
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)
        data, target = data.float(), target.float()
        optimizer.zero_grad()
        y_pred = mymodel(data)
        if y_pred.dim() == 3:
            y_pred = y_pred[:, -1, 0]
        elif y_pred.dim() == 2:
            y_pred = y_pred.view(y_pred.size()[0])
        loss = criterion(y_pred, target)
        loss.backward()
        optimizer.step()
        print("epoch: {}, loss: {}".format(i, loss.item()))

epoch: 0, loss: 0.7734392285346985
epoch: 1, loss: 0.7692630887031555
epoch: 2, loss: 0.7646950483322144
epoch: 3, loss: 0.7594479918479919
epoch: 4, loss: 0.752925455570221
epoch: 5, loss: 0.7437955141067505
epoch: 6, loss: 0.7328096628189087
epoch: 7, loss: 0.7192389369010925
epoch: 8, loss: 0.7033030986785889
epoch: 9, loss: 0.6819088459014893
epoch: 10, loss: 0.661637008190155
epoch: 11, loss: 0.6421570181846619
epoch: 12, loss: 0.6238018274307251
epoch: 13, loss: 0.6085578799247742
epoch: 14, loss: 0.5965955853462219
epoch: 15, loss: 0.586445689201355
epoch: 16, loss: 0.578726589679718
epoch: 17, loss: 0.5726451873779297
epoch: 18, loss: 0.5682634711265564
epoch: 19, loss: 0.5649642944335938
epoch: 20, loss: 0.562863826751709
epoch: 21, loss: 0.5601550936698914
epoch: 22, loss: 0.5597248077392578
epoch: 23, loss: 0.5577866435050964
epoch: 24, loss: 0.5570876002311707
epoch: 25, loss: 0.5554132461547852
epoch: 26, loss: 0.5548388361930847
epoch: 27, loss: 0.554685652256012
epoch: 2

epoch: 228, loss: 0.5468522906303406
epoch: 229, loss: 0.5468352437019348
epoch: 230, loss: 0.5468383431434631
epoch: 231, loss: 0.5468542575836182
epoch: 232, loss: 0.5468453168869019
epoch: 233, loss: 0.5468384027481079
epoch: 234, loss: 0.546837568283081
epoch: 235, loss: 0.5468310713768005
epoch: 236, loss: 0.5468349456787109
epoch: 237, loss: 0.5468649864196777
epoch: 238, loss: 0.5468291640281677
epoch: 239, loss: 0.5468319654464722
epoch: 240, loss: 0.5468374490737915
epoch: 241, loss: 0.5468349456787109
epoch: 242, loss: 0.5468350052833557
epoch: 243, loss: 0.5468422174453735
epoch: 244, loss: 0.546839714050293
epoch: 245, loss: 0.5468395948410034
epoch: 246, loss: 0.5468533635139465
epoch: 247, loss: 0.5468560457229614
epoch: 248, loss: 0.5468576550483704
epoch: 249, loss: 0.5468341112136841
epoch: 250, loss: 0.5468369126319885
epoch: 251, loss: 0.5468449592590332
epoch: 252, loss: 0.5468430519104004
epoch: 253, loss: 0.5468377470970154
epoch: 254, loss: 0.5468363165855408
epo

epoch: 454, loss: 0.5468299984931946
epoch: 455, loss: 0.5468302965164185
epoch: 456, loss: 0.5468224883079529
epoch: 457, loss: 0.5468516945838928
epoch: 458, loss: 0.5468341708183289
epoch: 459, loss: 0.5468247532844543
epoch: 460, loss: 0.5468302965164185
epoch: 461, loss: 0.5468232035636902
epoch: 462, loss: 0.5468230247497559
epoch: 463, loss: 0.5468224883079529
epoch: 464, loss: 0.5468317270278931
epoch: 465, loss: 0.5468360781669617
epoch: 466, loss: 0.5468422770500183
epoch: 467, loss: 0.5468260645866394
epoch: 468, loss: 0.5468267798423767
epoch: 469, loss: 0.5468310117721558
epoch: 470, loss: 0.5468253493309021
epoch: 471, loss: 0.5468268394470215
epoch: 472, loss: 0.5468299388885498
epoch: 473, loss: 0.5468271970748901
epoch: 474, loss: 0.5468432903289795
epoch: 475, loss: 0.5468283295631409
epoch: 476, loss: 0.5468222498893738
epoch: 477, loss: 0.5468335151672363
epoch: 478, loss: 0.5468297004699707
epoch: 479, loss: 0.5468374490737915
epoch: 480, loss: 0.5468277335166931
e

epoch: 680, loss: 0.5469223856925964
epoch: 681, loss: 0.5468239784240723
epoch: 682, loss: 0.5468214750289917
epoch: 683, loss: 0.5468225479125977
epoch: 684, loss: 0.5468304753303528
epoch: 685, loss: 0.546820342540741
epoch: 686, loss: 0.5468613505363464
epoch: 687, loss: 0.5468226671218872
epoch: 688, loss: 0.5468220114707947
epoch: 689, loss: 0.5468234419822693
epoch: 690, loss: 0.5468469858169556
epoch: 691, loss: 0.5468205213546753
epoch: 692, loss: 0.5468286275863647
epoch: 693, loss: 0.5468229055404663
epoch: 694, loss: 0.5468247532844543
epoch: 695, loss: 0.5468246340751648
epoch: 696, loss: 0.5468234419822693
epoch: 697, loss: 0.5468224287033081
epoch: 698, loss: 0.5468268990516663
epoch: 699, loss: 0.5468201637268066
epoch: 700, loss: 0.5468226075172424
epoch: 701, loss: 0.5468340516090393
epoch: 702, loss: 0.5468213558197021
epoch: 703, loss: 0.5468204617500305
epoch: 704, loss: 0.5468253493309021
epoch: 705, loss: 0.546820342540741
epoch: 706, loss: 0.5468232035636902
epo

epoch: 906, loss: 0.5468204021453857
epoch: 907, loss: 0.5468212962150574
epoch: 908, loss: 0.5468187928199768
epoch: 909, loss: 0.5468205809593201
epoch: 910, loss: 0.5468255281448364
epoch: 911, loss: 0.5468258261680603
epoch: 912, loss: 0.5468263030052185
epoch: 913, loss: 0.5468264818191528
epoch: 914, loss: 0.546820342540741
epoch: 915, loss: 0.5468190312385559
epoch: 916, loss: 0.5468201637268066
epoch: 917, loss: 0.546820342540741
epoch: 918, loss: 0.5468222498893738
epoch: 919, loss: 0.5468197464942932
epoch: 920, loss: 0.5468199253082275
epoch: 921, loss: 0.5468195676803589
epoch: 922, loss: 0.546819806098938
epoch: 923, loss: 0.546818733215332
epoch: 924, loss: 0.546819269657135
epoch: 925, loss: 0.5468189120292664
epoch: 926, loss: 0.5468270778656006
epoch: 927, loss: 0.5468223690986633
epoch: 928, loss: 0.5468185544013977
epoch: 929, loss: 0.5468196868896484
epoch: 930, loss: 0.5468233227729797
epoch: 931, loss: 0.5468195676803589
epoch: 932, loss: 0.5468209385871887
epoch:

In [32]:
import csv
mymodel.eval()
with open("submission.csv", 'w') as f:
    mywriter = csv.writer(f)
    mywriter.writerow(["vid_id", "is_turkey"])
    for idx, (data, vid) in enumerate(test_dataloader):
        data = Variable(data)
        data = data.to(device)
        data = data.float()
        y_pred = mymodel(data)
        y_pred = y_pred.view(y_pred.size()[0])
        print(y_pred)
        y_pred = y_pred.item()
        mywriter.writerow([vid[0], y_pred])

tensor([8.1861e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.7546e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0685e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4986e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.8587e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.8101e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.9323e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.2121e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4065e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.9376e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewB

tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.6567e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.1409e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.8276e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4436e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.8924e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.3607e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.6750e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([2.1624e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.1882e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.1645e-08], device='cuda:0', grad_fn=<V

tensor([1.2860e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4762e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.9192e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.1650e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([2.9211e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.2798e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0579e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0546e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.4610e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.4628e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0908e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.3127e-09], device='cuda:0', grad_f

tensor([0.0002], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.8097e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.6953e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.1713e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([2.0191e-06], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0373e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.1054e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.1249e-07], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4067e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.5332e-08], device='cuda:0', grad_fn=<ViewBackward>

tensor([1.1668e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.1985e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.7902e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.3637e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4501e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.7929e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.2157e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.3223e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.6241e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
te

tensor([7.3746e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.7134e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.1043e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.3186e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.2058e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0963e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.3498e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.7113e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
te

tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([0.0051], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.3627e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.3210e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.1600e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.1317e-08], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.7522e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.8854e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([0.9837], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.0000], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.1756e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([8.4573e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([1.], device='cuda:0', grad_fn=<ViewBackward>)
tensor([7.9817e-09], device='cuda:0', grad_fn=<ViewBackward>)
tensor([9.1997e-09], device='cuda:0', grad_fn=<ViewB